In [9]:
from datasets import load_from_disk
from transformers import OPTForCausalLM, AutoTokenizer, DataCollatorWithPadding

In [10]:
datasets = load_from_disk(r'data/galactica-125m/tokenized_applications.json')
checkpoint = 'checkpoint-14336'
model = OPTForCausalLM.from_pretrained('checkpoint-14336').base_model
tokenizer = AutoTokenizer.from_pretrained('checkpoint-14336')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at checkpoint-14336 were not used when initializing OPTForCausalLM: ['score.weight']
- This IS expected if you are initializing OPTForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing OPTForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
testset = datasets['test']
testset_ = testset.remove_columns(['_labels', 'id', 'title', 'text', 'token_type_ids', 'labels'])
testset_.set_format("torch")

In [12]:
embds = {'embds': [], 'labels': []}
batch_size = 64

for i in range(len(testset_)//batch_size):
    batch = testset_[i*batch_size:(i+1)*batch_size]
    labels = testset[i*batch_size:(i+1)*batch_size]['labels']
    batch = data_collator(batch)
    outputs = model(**batch)
    embds['embds'].append(outputs.last_hidden_state.mean(dim=1))
    embds['labels'].append(labels)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
